In [1]:
import numpy as np
import pandas as pd


from matplotlib import pyplot as plt
import seaborn as sns
from connectors import read_sql_query
import urllib3

urllib3.disable_warnings()
from typing import List, Optional, Tuple
import sqlalchemy
from urllib.parse import quote_plus
from datetime import datetime,timedelta
from tqdm import tqdm

Озерки

In [2]:
retailer = 333 #332 для доктора столетова
start_date = '2024-09-17'
end_date = '2024-09-18'

In [3]:
list_df = []
for date in tqdm(pd.date_range(start=start_date, end=end_date)):
    df = read_sql_query(f'''
    with all_scores_cte as (
    select
        dt as datetime_score_update, --дата+время обновления скора
        date(dt) as date_dt, --дата обновления скора
        store_id, --ID магазина
        retailer_id, --ID ритейлера
        retailer_sku, --СКЮ ритейлера
        round(cast(element_at(filter(data, x -> x.key = 'score'), 1)[2][1] as double),3) as score --собственно скор oos модели
    from iceberg.dwh_raw_producthub.raw_stock_2 
    where date(dt) between date('{date.strftime("%Y-%m-%d")}') and date('{date.strftime("%Y-%m-%d")}')
    and ab_group = '' 
    and retailer_id = {retailer}
    group by 1,2,3,4,5,6

    )

    select 
        retailer_sku, 
        retailer_id,  
        store_id, 
        datetime_score_update,
        date_dt,
        score
    from all_scores_cte
    where score is not null 
                            ''', con = 'trino')
    list_df.append(df)

100%|██████████| 2/2 [03:33<00:00, 106.89s/it]


In [4]:
df_scores = pd.concat(list_df)

In [5]:
#данные по метрикам целостности
df_rc_data = read_sql_query(f'''
with 
    damage_orders_items_cte as (
    select 
        shipment_number,
        sku,
        retailer_sku,
        sku_status,
        is_replacement,
        quantity,
        found_quantity,
        weight_added_items,
        weight_deleted_items,
        gmv_final
    from sandbox.damage_orders_items 
    where dt BETWEEN toDate('{start_date}') AND toDate('{end_date}')
    ),


   line_items_stats_cte as (
    select 
    shipment_number, 
    retailer_sku,
    countIf(sku, is_replacement = 0 and quantity > 0 ) as initial_sku,
    countIf(sku, sku_status in  ('replaced', 'cancelled')) as rc_count_sku,
    countIf(sku, sku_status in ('cancelled'))  as canc_count_sku,
    countIf(sku, sku_status in ('replaced'))  as repl_count_sku,
    sumIf(quantity,  is_replacement = 0) as initial_items,
    sumIf(found_quantity,  sku_status not in  ('replaced', 'cancelled')) as final_items,
    sum(weight_added_items) as weight_added_items,
    sum(weight_deleted_items) as weight_deleted_items,
    sum(gmv_final) as gmv_final
    from damage_orders_items_cte 
    group by 1,2
    ),
    
    shipments_stats_cte as (
    select 
    shipment_id as shipment_id, 
    shipment_number as shipment_number,
    completed_at,
    starts_at,
    store_id as store_id,
    store_name, 
    retailer_id,
    retailer_name,
    type_delivery
    from analytics.bi_shipments_financial
    where toDate(completed_at) BETWEEN toDate('{start_date}') AND toDate('{end_date}')
    and shipment_state = 'shipped'
    and retailer_id = {retailer}
    )

    select 
        toDate(starts_at) as starts_at_date,
        shipment_id,
        starts_at,
        retailer_sku,
        store_id,
        store_name,
        retailer_id,
        retailer_name,
        gmv_final,
        ifNull(initial_sku, 0) as initial_sku, 
        ifNull(rc_count_sku, 0) as  rc_count_sku, 
        ifNull(canc_count_sku, 0) as  canc_count_sku, 
        ifNull(repl_count_sku, 0) as  repl_count_sku, 
        ifNull(initial_items, 0) as  initial_items,
        ifNull(if(type_delivery = 'scan_pay_go', initial_items, final_items),0) as final_items, 
        ifNull(weight_added_items,0) as weight_added_items, --добавленное кол-во штук
        ifNull(if(type_delivery = 'scan_pay_go', 0, weight_deleted_items),0) as weight_deleted_items, --удаленное кол-во штук
        sum(initial_items) over(partition by shipment_number) as initial_items_sum, --сумма всех изначальных товаров
        sum(weight_added_items) over(partition by shipment_number) as weight_added_items_sum, --сумма всех добавленных товаров в заказе
        sum(weight_deleted_items) over(partition by shipment_number) as weight_deleted_items_sum --сумма всех удаленных товаров в заказе
    from line_items_stats_cte 
    inner join shipments_stats_cte
        on line_items_stats_cte.shipment_number = shipments_stats_cte.shipment_number
                            ''')

In [6]:
df_scores['datetime_score_update'] = df_scores['datetime_score_update'].apply(lambda x: str(x)[0:19])
df_scores['datetime_score_update'] = pd.to_datetime(df_scores['datetime_score_update'])
df_scores['date_dt'] = pd.to_datetime(df_scores['date_dt'], format='%Y-%m-%d')

df_scores['retailer_sku'] = df_scores['retailer_sku'].astype('str')
df_rc_data['retailer_sku'] = df_rc_data['retailer_sku'].astype('str')

merged_df = df_rc_data.merge(df_scores, how = 'left', on = ['retailer_sku', 'store_id'])
merged_df_f = merged_df[merged_df.score.notnull()].query('starts_at > datetime_score_update') #выбираем время, где время старта слота больше времени обновления скора
merged_df_f = merged_df_f.drop(['retailer_id_x', 'retailer_id_y'], axis=1)

#берем максимальное время из отфильтрованных ранее
max_update = pd.DataFrame(merged_df_f.groupby(['retailer_sku', 'store_id', 'starts_at']).datetime_score_update.max()).reset_index()

#отбираем те строки со скором, где время обновления скора равно максимальному ДО времени начала слота
df_scores_final = merged_df_f.merge(max_update, how = 'left', on = ['retailer_sku', 'store_id', 'starts_at']) \
        .query('datetime_score_update_x == datetime_score_update_y')\
        .drop(['datetime_score_update_x'], axis=1) \
        .rename(columns={"datetime_score_update_y": "datetime_score_update_fin"})


In [28]:
threshold = 0.17 #выбираем порог, который установим

In [29]:
#текущие отмены/замены
current_rc = df_scores_final.rc_count_sku.sum()/df_scores_final.initial_sku.sum()*100
current_rc

24.051654560129137

In [30]:
#Потенциальное улучшение отмен/замен
df_scores_final['rc_count_sku_new'] = np.where((df_scores_final['score'] < 0.24) & (df_scores_final['score'] >= threshold), 0, df_scores_final['rc_count_sku'])
a = df_scores_final.rc_count_sku_new.sum()/df_scores_final.initial_sku.sum()*100
b = df_scores_final.rc_count_sku.sum()/df_scores_final.initial_sku.sum()*100
new_rc = a/b*100-100
new_rc

-1.3422818791946298

In [31]:
current_rc + current_rc/100*new_rc

23.728813559322035

In [11]:
#снэпшоты с данными по доступности товарного ассортимента, 1 на одну дату, без изменений в течение дня
availability_data = read_sql_query(f'''
select 
	retailer_sku, 
	product_sku,
	retailer_id,
	store_id,
    case when published = true then 1 else 0 end as published,
    published_reason,                            
	snapshot_date
from iceberg.dwh_ods_producthub.stocks_history 
where date(snapshot_date) between date('{start_date}') and date('{end_date}')
and retailer_id = {retailer}
                            ''', con = 'trino'

    )

In [12]:
availability_data['snapshot_date'] = pd.to_datetime(availability_data['snapshot_date'], format='%Y-%m-%d')
availability_data['retailer_sku'] = availability_data['retailer_sku'].astype('str')

merged_avail_df = availability_data.merge(df_scores, how = 'left', on = ['retailer_sku', 'store_id'])
merged_avail_df = merged_avail_df[merged_avail_df.score.notnull()].query('snapshot_date >= date_dt') #отбираем строки, где дата обновления скора равна дате снэпшота 
merged_avail_df = merged_avail_df.drop(['retailer_id_x', 'retailer_id_y'], axis=1)

#отбираем самый последний сток на дату (можно выбирать самый первый, и то, и то будет не до конца корректно, но нам нужно выбрать)
max_update_avail = pd.DataFrame(merged_avail_df.groupby(['retailer_sku', 'store_id', 'snapshot_date']).datetime_score_update.max()).reset_index()

#оставляем только те строки, где время обновления скора равно максимальному времени обновления скора в этот день
avail_df_final = merged_avail_df.merge(max_update_avail, how = 'left', on = ['retailer_sku', 'store_id', 'snapshot_date']) \
        .query('datetime_score_update_x == datetime_score_update_y')\
        .drop(['datetime_score_update_x'], axis=1) \
        .rename(columns={"datetime_score_update_y": "datetime_score_update_fin"})

In [32]:
current_avail = avail_df_final.published.sum()/avail_df_final.published.count()*100
current_avail

98.63164866402904

In [33]:
#Потенциальное СНИЖЕНИЕ доступности
avail_df_final['published_new'] = np.where((avail_df_final['score'] < 0.24) & (avail_df_final['score'] >= threshold) & (avail_df_final['published_reason'] == 'on-by-oos') \
                                            , 0, avail_df_final['published'])
a = avail_df_final.published_new.sum()/avail_df_final.published.count()*100
b = avail_df_final.published.sum()/avail_df_final.published.count()*100
new_avail = a/b*100-100
new_avail

-9.978707827741488

In [35]:
new_avail_number = current_avail + current_avail/100*new_avail
new_avail_number

88.7894846181611

In [36]:
new_avail_number - current_avail

-9.842164045867946

Доктор столетов

In [2]:
retailer = 332 #332 для доктора столетова
threshold = 0.20 #выбираем порог, который установим
start_date = '2024-09-16'
end_date = '2024-09-18'

In [3]:
list_df = []
for date in tqdm(pd.date_range(start=start_date, end=end_date)):
    df = read_sql_query(f'''
    with all_scores_cte as (
    select
        dt as datetime_score_update, --дата+время обновления скора
        date(dt) as date_dt, --дата обновления скора
        store_id, --ID магазина
        retailer_id, --ID ритейлера
        retailer_sku, --СКЮ ритейлера
        round(cast(element_at(filter(data, x -> x.key = 'score'), 1)[2][1] as double),3) as score --собственно скор oos модели
    from iceberg.dwh_raw_producthub.raw_stock_2 
    where date(dt) between date('{date.strftime("%Y-%m-%d")}') and date('{date.strftime("%Y-%m-%d")}')
    and ab_group = '' 
    and retailer_id = {retailer}
    group by 1,2,3,4,5,6

    )

    select 
        retailer_sku, 
        retailer_id,  
        store_id, 
        datetime_score_update,
        date_dt,
        score
    from all_scores_cte
    where score is not null 
                            ''', con = 'trino')
    list_df.append(df)

100%|██████████| 3/3 [06:51<00:00, 137.29s/it]


In [4]:
df_scores = pd.concat(list_df)

In [5]:
#данные по метрикам целостности
df_rc_data = read_sql_query(f'''
with 
    damage_orders_items_cte as (
    select 
        shipment_number,
        sku,
        retailer_sku,
        sku_status,
        is_replacement,
        quantity,
        found_quantity,
        weight_added_items,
        weight_deleted_items,
        gmv_final
    from sandbox.damage_orders_items 
    where dt BETWEEN toDate('{start_date}') AND toDate('{end_date}')
    ),


   line_items_stats_cte as (
    select 
    shipment_number, 
    retailer_sku,
    countIf(sku, is_replacement = 0 and quantity > 0 ) as initial_sku,
    countIf(sku, sku_status in  ('replaced', 'cancelled')) as rc_count_sku,
    countIf(sku, sku_status in ('cancelled'))  as canc_count_sku,
    countIf(sku, sku_status in ('replaced'))  as repl_count_sku,
    sumIf(quantity,  is_replacement = 0) as initial_items,
    sumIf(found_quantity,  sku_status not in  ('replaced', 'cancelled')) as final_items,
    sum(weight_added_items) as weight_added_items,
    sum(weight_deleted_items) as weight_deleted_items,
    sum(gmv_final) as gmv_final
    from damage_orders_items_cte 
    group by 1,2
    ),
    
    shipments_stats_cte as (
    select 
    shipment_id as shipment_id, 
    shipment_number as shipment_number,
    completed_at,
    starts_at,
    store_id as store_id,
    store_name, 
    retailer_id,
    retailer_name,
    type_delivery
    from analytics.bi_shipments_financial
    where toDate(completed_at) BETWEEN toDate('{start_date}') AND toDate('{end_date}')
    and shipment_state = 'shipped'
    and retailer_id = {retailer}
    )

    select 
        toDate(starts_at) as starts_at_date,
        shipment_id,
        starts_at,
        retailer_sku,
        store_id,
        store_name,
        retailer_id,
        retailer_name,
        gmv_final,
        ifNull(initial_sku, 0) as initial_sku, 
        ifNull(rc_count_sku, 0) as  rc_count_sku, 
        ifNull(canc_count_sku, 0) as  canc_count_sku, 
        ifNull(repl_count_sku, 0) as  repl_count_sku, 
        ifNull(initial_items, 0) as  initial_items,
        ifNull(if(type_delivery = 'scan_pay_go', initial_items, final_items),0) as final_items, 
        ifNull(weight_added_items,0) as weight_added_items, --добавленное кол-во штук
        ifNull(if(type_delivery = 'scan_pay_go', 0, weight_deleted_items),0) as weight_deleted_items, --удаленное кол-во штук
        sum(initial_items) over(partition by shipment_number) as initial_items_sum, --сумма всех изначальных товаров
        sum(weight_added_items) over(partition by shipment_number) as weight_added_items_sum, --сумма всех добавленных товаров в заказе
        sum(weight_deleted_items) over(partition by shipment_number) as weight_deleted_items_sum --сумма всех удаленных товаров в заказе
    from line_items_stats_cte 
    inner join shipments_stats_cte
        on line_items_stats_cte.shipment_number = shipments_stats_cte.shipment_number
                            ''')

In [6]:
df_scores['datetime_score_update'] = df_scores['datetime_score_update'].apply(lambda x: str(x)[0:19])
df_scores['datetime_score_update'] = pd.to_datetime(df_scores['datetime_score_update'])
df_scores['date_dt'] = pd.to_datetime(df_scores['date_dt'], format='%Y-%m-%d')

df_scores['retailer_sku'] = df_scores['retailer_sku'].astype('str')
df_rc_data['retailer_sku'] = df_rc_data['retailer_sku'].astype('str')

merged_df = df_rc_data.merge(df_scores, how = 'left', on = ['retailer_sku', 'store_id'])
merged_df_f = merged_df[merged_df.score.notnull()].query('starts_at > datetime_score_update') #выбираем время, где время старта слота больше времени обновления скора
merged_df_f = merged_df_f.drop(['retailer_id_x', 'retailer_id_y'], axis=1)

#берем максимальное время из отфильтрованных ранее
max_update = pd.DataFrame(merged_df_f.groupby(['retailer_sku', 'store_id', 'starts_at']).datetime_score_update.max()).reset_index()

#отбираем те строки со скором, где время обновления скора равно максимальному ДО времени начала слота
df_scores_final = merged_df_f.merge(max_update, how = 'left', on = ['retailer_sku', 'store_id', 'starts_at']) \
        .query('datetime_score_update_x == datetime_score_update_y')\
        .drop(['datetime_score_update_x'], axis=1) \
        .rename(columns={"datetime_score_update_y": "datetime_score_update_fin"})

In [7]:
threshold = 0.17 #выбираем порог, который установим

In [8]:
#текущие отмены/замены
current_rc = df_scores_final.rc_count_sku.sum()/df_scores_final.initial_sku.sum()*100
current_rc

17.005545286506468

In [19]:
#Потенциальное улучшение отмен/замен
df_scores_final['rc_count_sku_new'] = np.where((df_scores_final['score'] < 0.24) & (df_scores_final['score'] >= threshold), 0, df_scores_final['rc_count_sku'])
a = df_scores_final.rc_count_sku_new.sum()/df_scores_final.initial_sku.sum()*100
b = df_scores_final.rc_count_sku.sum()/df_scores_final.initial_sku.sum()*100
new_rc = a/b*100-100
new_rc

-8.15217391304347

In [18]:
current_rc + current_rc/100*new_rc

17.13995943204868

In [11]:
#снэпшоты с данными по доступности товарного ассортимента, 1 на одну дату, без изменений в течение дня
availability_data = read_sql_query(f'''
select 
	retailer_sku, 
	product_sku,
	retailer_id,
	store_id,
    case when published = true then 1 else 0 end as published,
    published_reason,                            
	snapshot_date
from iceberg.dwh_ods_producthub.stocks_history 
where date(snapshot_date) between date('{start_date}') and date('{end_date}')
and retailer_id = {retailer}
                            ''', con = 'trino'

    )

In [12]:
availability_data['snapshot_date'] = pd.to_datetime(availability_data['snapshot_date'], format='%Y-%m-%d')
availability_data['retailer_sku'] = availability_data['retailer_sku'].astype('str')

merged_avail_df = availability_data.merge(df_scores, how = 'left', on = ['retailer_sku', 'store_id'])
merged_avail_df = merged_avail_df[merged_avail_df.score.notnull()].query('snapshot_date >= date_dt') #отбираем строки, где дата обновления скора равна дате снэпшота 
merged_avail_df = merged_avail_df.drop(['retailer_id_x', 'retailer_id_y'], axis=1)

#отбираем самый последний сток на дату (можно выбирать самый первый, и то, и то будет не до конца корректно, но нам нужно выбрать)
max_update_avail = pd.DataFrame(merged_avail_df.groupby(['retailer_sku', 'store_id', 'snapshot_date']).datetime_score_update.max()).reset_index()

#оставляем только те строки, где время обновления скора равно максимальному времени обновления скора в этот день
avail_df_final = merged_avail_df.merge(max_update_avail, how = 'left', on = ['retailer_sku', 'store_id', 'snapshot_date']) \
        .query('datetime_score_update_x == datetime_score_update_y')\
        .drop(['datetime_score_update_x'], axis=1) \
        .rename(columns={"datetime_score_update_y": "datetime_score_update_fin"})

In [13]:
current_avail = avail_df_final.published.sum()/avail_df_final.published.count()*100
current_avail

98.94917042066848

In [14]:
#Потенциальное СНИЖЕНИЕ доступности
avail_df_final['published_new'] = np.where((avail_df_final['score'] < 0.24) & (avail_df_final['score'] >= threshold) & (avail_df_final['published_reason'] == 'on-by-oos') \
                                            , 0, avail_df_final['published'])
a = avail_df_final.published_new.sum()/avail_df_final.published.count()*100
b = avail_df_final.published.sum()/avail_df_final.published.count()*100
new_avail = a/b*100-100
new_avail

-10.172566957113418

In [15]:
new_avail_number = current_avail + current_avail/100*new_avail
new_avail_number

88.88349980611771

In [16]:
new_avail_number - current_avail

-10.065670614550768